# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key\
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,0.67,97,100,3.46,GL,1702361560
1,1,bairnsdale,-37.8333,147.6167,22.81,81,54,2.24,AU,1702361561
2,2,ciudad lazaro cardenas,17.9583,-102.2000,24.55,61,5,1.96,MX,1702361561
3,3,papatowai,-46.5619,169.4708,9.21,74,75,9.06,NZ,1702361562
4,4,port alfred,-33.5906,26.8910,21.21,78,18,3.13,ZA,1702361563


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:


# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size=10, hover_cols=['City'], tiles = 'OSM', color = 'purple')

# Display the map
map_plot.opts(global_extent=True, width=700, height=400)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values

min_temp_celsius = (65 - 32) * 5.0/9.0
max_temp_celsius = (85 - 32) * 5.0/9.0

temp_filtered_df = city_data_df[
    (city_data_df['Max Temp'] <= max_temp_celsius) & 
    (city_data_df['Max Temp'] >= min_temp_celsius)
]

# Drop any rows with null values
temp_filtered_df = temp_filtered_df.dropna()
# Display sample data
temp_filtered_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,bairnsdale,-37.8333,147.6167,22.81,81,54,2.24,AU,1702361561
2,2,ciudad lazaro cardenas,17.9583,-102.2000,24.55,61,5,1.96,MX,1702361561
4,4,port alfred,-33.5906,26.8910,21.21,78,18,3.13,ZA,1702361563
8,8,blackmans bay,-43.0167,147.3167,21.52,50,100,0.89,AU,1702361565
17,17,baoro,5.6667,15.9667,20.46,32,99,2.66,CF,1702361570
...,...,...,...,...,...,...,...,...,...,...
581,581,almeirim,-1.5233,-52.5817,23.49,95,98,1.41,BR,1702361928
582,582,bikenibeu village,1.3673,173.1241,28.40,73,100,2.24,KI,1702361929
583,583,puerto baquerizo moreno,-0.9000,-89.6000,23.95,87,62,3.55,EC,1702361930
584,584,namdrik,5.5994,168.1028,27.05,83,100,7.18,MH,1702361930


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = temp_filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
1,bairnsdale,AU,-37.8333,147.6167,81,
2,ciudad lazaro cardenas,MX,17.9583,-102.2000,61,
4,port alfred,ZA,-33.5906,26.8910,78,
8,blackmans bay,AU,-43.0167,147.3167,50,
17,baoro,CF,5.6667,15.9667,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # Search within 10,000 meters
params = {
    "apiKey": geoapify_key, # API key
    "categories": "accommodation.hotel",  # Specify the category of the place as hotel
    "limit": 1,  # Limit to one result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"

    # Make an API request using the params dictionary
    response = requests.get("https://api.geoapify.com/v2/places", params=params)
    
    # Check if the response was successful before continuing
    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()
        
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found"
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        print(f"Failed to get a response for the city: {row['City']}, status code: {response.status_code}")
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df.head())


Starting hotel search
bairnsdale - nearest hotel: The Wy Yung Pub
ciudad lazaro cardenas - nearest hotel: Quinta Antigua Hotel
port alfred - nearest hotel: Ferryman
blackmans bay - nearest hotel: Kingston Hotel
baoro - nearest hotel: No hotel found
yeppoon - nearest hotel: Blossom's on Seaspray
tofol - nearest hotel: Nautilus Resort
chiredzi - nearest hotel: King Fisher lodge
tura - nearest hotel: No hotel found
saipan - nearest hotel: Hyatt Regency Saipan
queenstown - nearest hotel: Queens Hotel
margaret river - nearest hotel: Quality Inn Margaret River
gemena - nearest hotel: La Joie
kavaratti - nearest hotel: No hotel found
sao joao da barra - nearest hotel: Unidade Hoteleira Sesc Grussaí
atafu village - nearest hotel: No hotel found
saint-joseph - nearest hotel: PALM Hotel & Spa
san jose de guanipa - nearest hotel: Premier Hotel
fale old settlement - nearest hotel: No hotel found
nova vicosa - nearest hotel: Pousada Pontal da Barra
bonthe - nearest hotel: No hotel found
faaa - near

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:


# Configure the map plot
hotels_found_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]


map_with_hotels = hotels_found_df.hvplot.points(
    'Lng', 'Lat', geo=True, size=10,
    hover_cols=['City', 'Hotel Name', 'Country'], tiles = 'OSM', color = 'purple'
)

# Display the map
map_with_hotels.opts(global_extent=True, width=700, height=400)



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)